## Задание 3

В данном упражнении вам предстоит решить задачу **многокласовой классификации**, используя в качестве тренировочного набора данных — набор данных `MNIST`, содержащий образы рукописных цифр.

Изображения в наборе данных MNIST имеют размер 28 на 28 пикселей и выполнены в оттенках серого цвета. Большинство инструментов уже содержат этот набор данных в удобном для обработки формате.

Самый простой способ — это загрузить данные из библиотеки `keras.datasets`, где каждое изображение представлено в формате двумерного массива размера 28 на 28 и массива откликов.

In [124]:
from keras.datasets import mnist

(X_train, y_train), (X_pred, y_pred) = mnist.load_data()

In [125]:
X_train[0].shape

(28, 28)

In [126]:
from numpy import reshape

X_train_reshape = reshape(X_train, (60000, 784))
X_train_reshape.shape

(60000, 784)

Используйте метод главных компонент для набора данных `MNIST` (train dataset объема 60000).

Определите, какое минимальное количество главных компонент необходимо использовать, чтобы доля объясненной дисперсии превышала `0.83` (была строго больше указанного значения).

In [128]:
from sklearn.decomposition import PCA

for i in range(51, 53):
  print(f'for n_components={i}:', sum(PCA(n_components=i).fit(X_train_reshape).explained_variance_ratio_))

for n_components=51: 0.8277953049629361
for n_components=52: 0.8309381656784669


Введите количество главных компонент M:

Ответ: 52

Найдите счеты, отвечающие найденным в предыдущем пункте главным компонентам: `PCA(n_components=M, svd_solver='full')`

In [113]:
M = 52
pca = PCA(n_components=M, svd_solver='full')
X_train_reduced = pca.fit_transform(X_train_reshape)
X_train_reduced.shape

(60000, 52)

Разделите полученную выборку (после сокращения размерности) случайным образом в отношении 70/30: `train_test_split(X_train_reduced, y_train, test_size=0.3, random_state=126)`

In [129]:
from sklearn.model_selection import train_test_split

X_tr, X_t, y_tr, y_t = train_test_split(X_train_reduced, y_train, test_size=0.3, random_state=126)

Введите выборочное среднее нулевой колонки для полученного тренировочного набора:

In [130]:
from numpy import mean

round(mean(X_tr[:,0]), 2)

0.6

Используя метод **многоклассовой классификации One-vs-All** `OneVsRestClassifier()`, обучите алгоритм случайного леса `RandomForestClassifier()` с параметрами `criterion='gini', min_samples_leaf=10, max_depth=20, n_estimators=10, random_state=126`. Выполните оценку с помощью тестовых данных.

In [131]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

forest = RandomForestClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, n_estimators=10, random_state=126)
clf = OneVsRestClassifier(forest).fit(X_tr, y_tr)

In [133]:
from numpy import ndenumerate

class_5_id = [i[0] for i, digit in ndenumerate(y_t) if digit == 5]
len(class_5_id)

1610

In [134]:
from numpy import full

clf.score(X_t[class_5_id], full(len(class_5_id), 5)) * len(class_5_id)

1414.0

Примените полученное ранее преобразование метода главных компонент к [новым данным](https://courses.openedu.ru/assets/courseware/v1/25c0abab1076236b7427fac937c5663e/asset-v1:ITMOUniversity+ADVMLAUTXT+spring_2023_ITMO_mag+type@asset+block/pred_for_task.csv) (идентификаторы строк соответствуют файлам изображений).

In [135]:
import pandas as pd

df_test = pd.read_csv('https://courses.openedu.ru/assets/courseware/v1/25c0abab1076236b7427fac937c5663e/asset-v1:ITMOUniversity+ADVMLAUTXT+spring_2023_ITMO_mag+type@asset+block/pred_for_task.csv')
df_test.shape

(30, 786)

In [137]:
X_test = df_test.drop(columns=['FileName', 'Label'])
X_test_reduced = pca.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(


Выполните предсказание для указанных изображений цифр с помощью обученных алгоритмов.

In [138]:
y_test = clf.predict_proba(X_test_reduced)

Введите вероятность отнесения изображения `file20` к классу `4` алгоритмом случайного леса.

In [141]:
round(y_test[19][4], 3)

0.963